# <center> Lending Club Case Study


## Table of Content

1. [Introduction](#Introduction)
2. [Important Settings](#Important-Settings)
3. [Data Understanding](#Data-Understanding)  
4. [Data Cleaning and Manipulation](#Data-Cleaning-and-Manipulation)
    1. [Data Cleaning](#Data-Cleaning)
    2. [Data Manipulation](#Data-Manipulation)
5. [Data Analysis](#Data-Analysis)
6. [Presentation and Recommendations](#Presentation-and-Recommendations)
7. [Conciseness and Readability of the Code](#Conciseness-and-Readability-of-the-Code)


# Introduction 

### Group members:
* Pavan Narra (https://github.com/pavan440)
* Jheser Guzman (https://github.com/dicotips)

## Business Understanding

**Source:** UpGrad Assignment description

*You work for a **consumer finance** company which specialises in lending various types of loans to urban customers. When the company receives a loan application, the company has to make a decision for loan approval based on the applicant’s profile. Two **types of risks** are associated with the bank’s decision:*

* *If the applicant is **likely to repay the loan**, then not approving the loan results in a **loss of business** to the company*

* *If the applicant is **not likely to repay the loan**, i.e. he/she is likely to default, then approving the loan may lead to a **financial loss** for the company*

*The data given below contains the information about past loan applicants and whether they ‘defaulted’ or not. The aim is to identify patterns which indicate if a person is likely to default, which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc.*

When a person applies for a loan, there are two types of decisions that could be taken by the company:

1. **Loan accepted:** If the company approves the loan, there are 3 possible scenarios described below:

  * **Fully paid:** Applicant has fully paid the loan (the principal and the interest rate)

  * **Current:** Applicant is in the process of paying the instalments, i.e. the tenure of the loan is not yet completed. These candidates are not labelled as 'defaulted'.

  * **Charged-off:** Applicant has not paid the instalments in due time for a long period of time, i.e. he/she has defaulted on the loan 

2. **Loan rejected:** The company had rejected the loan (because the candidate does not meet their requirements etc.). Since the loan was rejected, there is no transactional history of those applicants with the company and so this data is not available with the company (and thus in this dataset)



# Important Settings

In [131]:
# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

#Ignore version warnings
import warnings
warnings.filterwarnings('ignore')

#Set Dataset File Path
DATA_FILE_PATH = '_dataset/loan.csv'

In [132]:
# Reading CSV file with dtype object and saving it in raw_data dataframe. 
# All the processing in the data will be stored in new dataframes

raw_data = pd.read_csv(DATA_FILE_PATH, dtype=object)

## Data Understanding
* All data quality issues are correctly identified and reported. 
* Wherever required, the meanings of the variables are correctly interpreted and written either in the comments or text.

In [133]:
# Getting the first 5 rows from the raw_data dataframe for data exploration
raw_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN


In [134]:
# Understanding the structure of the dataset
# Checking shape & datatype of raw_data dataframe
print(raw_data.shape)
print(raw_data.info())

(39717, 111)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: object(111)
memory usage: 33.6+ MB
None


In [135]:
print(raw_data.dtypes)

id                            object
member_id                     object
loan_amnt                     object
funded_amnt                   object
funded_amnt_inv               object
                               ...  
tax_liens                     object
tot_hi_cred_lim               object
total_bal_ex_mort             object
total_bc_limit                object
total_il_high_credit_limit    object
Length: 111, dtype: object


In [136]:
# We verify if there is any dublicated rows in ['id', 'member_id']

print(raw_data.duplicated(subset=None, keep='first').count())
sum(raw_data.duplicated(['id']))

## Result: There are no duplicated IDs !!!

39717


0

In [137]:
print(raw_data.duplicated(subset=None, keep='first').count())
sum(raw_data.duplicated(['member_id']))

## Result: There are no duplicated member_ids !!!

39717


0

In [138]:
# Checking how many rows have all missing values
empty_rows = raw_data.isnull().all(axis=1).sum()
print(f'N Empty Rows: {empty_rows}')

## Result: There are 54 columns have all missing values.

N Empty Rows: 0


In [139]:
# Checking how many columns have all missing values
empty_columns = raw_data.isnull().all(axis=0).sum()
print(f'N Empty Columns: {empty_columns}')

## Result: There are 54 columns with no data in it (all data is null in those columns).

N Empty Columns: 54


In [175]:
# Counting Nulls in each column
raw_data.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

## Data Cleaning and Manipulation

* Data quality issues are addressed in the right way (missing value imputation, outlier treatment and other kinds of data redundancies, etc.). 
* If applicable, data is converted to a suitable and convenient format to work with using the right methods.
* Manipulation of strings and dates is done correctly wherever required.

### Data Cleaning

In [186]:
# Verifying how much memory is used by raw_data dataframe
raw_data_size_mb = format(sys.getsizeof(raw_data) /(1028**2), '.0f')

print(f'Raw_Data Memory Usage: {raw_data_size_mb}MB')

Raw_Data Memory Usage: 207MB


In [187]:
# Creating a copy of 'raw_data' to clean & transform data
## Dropping columns with all null values
columuns_all_null = raw_data.columns[raw_data.isnull().sum() == raw_data.shape[0]]
print(columuns_all_null)

cleaned_data = raw_data.drop(columns=columuns_all_null) 

Index(['mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m',
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
       'pct_tl_nvr_dl

In [188]:
# Counting Nulls in each column

cleaned_data.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_title                      2459
emp_length                     1075
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
url                               0
desc                          12940
purpose                           0
title                            11
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths              

In [189]:
# Droping additional columns we don't need since these are mostly nulls.
## Creating a list of Column names to be dropped
drop_columns = []

# * 'desc' is a plain text of description. It will not needed in this analysis
drop_columns.append('desc')
print(drop_columns)
cleaned_data.desc.head()

['desc']


0      Borrower added on 12/22/11 > I need to upgra...
1      Borrower added on 12/22/11 > I plan to use t...
2                                                  NaN
3      Borrower added on 12/21/11 > to pay for prop...
4      Borrower added on 12/21/11 > I plan on combi...
Name: desc, dtype: object

In [190]:
# Dropping following columns since they have too many Null values.
drop_columns.extend(["mths_since_last_delinq","mths_since_last_record","next_pymnt_d"])
print(cleaned_data.mths_since_last_delinq.value_counts())
print(cleaned_data.mths_since_last_record.value_counts())
print(cleaned_data.next_pymnt_d.value_counts())

0      443
15     252
23     247
30     247
24     241
      ... 
95       1
120      1
97       1
107      1
106      1
Name: mths_since_last_delinq, Length: 95, dtype: int64
0      670
104     61
89      60
113     59
111     57
      ... 
6        1
7        1
20       1
120      1
12       1
Name: mths_since_last_record, Length: 111, dtype: int64
Jun-16    1125
Jul-16      15
Name: next_pymnt_d, dtype: int64


In [191]:
# Dropping 
cleaned_data.isnull().sum()
print(cleaned_data.shape)

(39717, 57)


In [192]:
cleaned_data.drop(columns=drop_columns, inplace=True)
print(cleaned_data.shape)

(39717, 53)


In [226]:
# Dropping columns with all different values. They will not add value to the analysis.
## Dropping 'id' column
drop_columns = ['id']
print('id different values: ', end='')
print(cleaned_data.id.value_counts().count())

## Dropping 'member_id' column
drop_columns.append('member_id')
print('member_id different values: ', end='')
print(cleaned_data.member_id.value_counts().count())

## Dropping 'url' column since it has all different values
drop_columns.append('url')
print('url different values: ' , end='')
print(cleaned_data.url.value_counts().count())

id different values: 39717
member_id different values: 39717
url different values: 39717


In [227]:
# Dropping 'tax_liens' column since it has only one value 0
drop_columns.append('tax_liens')
print('tax_liens value count: ', end='')
print(cleaned_data.tax_liens.value_counts())

# Dropping 'emp_title' since it has too many different discrete values
drop_columns.append('emp_title')
print('emp_title different values: ' , end='')
print(cleaned_data.emp_title.value_counts().count())

# Dropping 'zip_code' since the dataset is too small for the universe of zip_codes in the country
drop_columns.append('zip_code')

tax_liens value count: 0    39678
Name: tax_liens, dtype: int64
emp_title different values: 28820


['id', 'member_id', 'url', 'tax_liens', 'emp_title', 'zip_code']

In [228]:
cleaned_data.drop(columns=drop_columns, inplace=True)

In [231]:
# After cleaning all the irrelevant columns for the analysis, we are left with a dataframe of 47 columns

cleaned_data.shape

(39717, 47)

In [233]:
# Verifying how much memory is used by cleaned_data dataframe
cleaned_data_size_mb = format(sys.getsizeof(cleaned_data) /(1028**2), '.0f')

print(f'Cleaned_Data Memory Usage: {cleaned_data_size_mb}MB')

## We are using almost half of the space in memory compared against raw_data

Cleaned_Data Memory Usage: 109MB


### Data Manipulation

## Data Analysis

* The right problem is solved which is coherent with the needs of the business. The analysis has a clear structure and the flow is easy to understand.
* Univariate and segmented univariate analysis is done correctly and appropriate realistic assumptions are made wherever required. The analyses successfully identify at least the 5 important driver variables (i.e. variables which are strong indicators of default).
* Business-driven, type-driven and data-driven metrics are created for the important variables and utilised for analysis. The explanation for creating the derived metrics is mentioned and is reasonable.
* Bivariate analysis is performed correctly and is able to identify the important combinations of driver variables. The combinations of variables are chosen such that they make business or analytical sense. 
* The most useful insights are explained correctly in the comments.
* Appropriate plots are created to present the results of the analysis. The choice of plots for respective cases is correct. The plots should clearly present the relevant insights and should be easy to read. The axes and important data points are labelled correctly.

## Presentation and Recommendations

* The presentation has a clear structure, is not too long, and explains the most important results concisely in simple language.
* The recommendations to solve the problems are realistic, actionable and coherent with the analysis. 
* The GitHub repository link contains a python file, presentation file and a README.md file. README.md file should describe the project briefly.
* If any assumptions are made, they are stated clearly.

## Conciseness and Readability of the Code
* The code is concise and syntactically correct. Wherever appropriate, built-in functions and standard libraries are used instead of writing long code (if-else statements, for loops, etc.).
* Custom functions are used to perform repetitive tasks.
* The code is readable with appropriately named variables and detailed comments are written wherever necessary.